# This code works but it gives just N/a in years , durations, rated

# Web scrapping of new data

To enhance the dataset, I chose to scrape additional data from the official IMDb website. The original dataset covered movies up to December 15, 2023, while the new dataset extends from December 16, 2023, to the present day (February 18, 2024).

I extracted the same set of information from the new dataset to maintain consistency and simplify the data cleaning and modification process. Subsequently, I will create a new Jupyter notebook to combine the two datasets and repeat all subsequent steps.

The data contains the following columns:
Moive Name, Rating, Votes, Meta Score, Genre, PG Rating, Year, Duration, Cast, Director

The link for getting data:
Moive Name, Rating, Votes, Meta Score, PG Rating, Year, Duration
https://www.imdb.com/search/title/?title_type=feature&release_date=2023-12-16,&primary_language=en

Other links are created in code to get movies' information of their:
Genre, Cast, Director

*IMDb allows users to use its content for non-personal use.

In [7]:
from bs4 import BeautifulSoup 
import sys
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

In [8]:
# Set up the Selenium webdriver
# we open Google Chrome
driver = webdriver.Chrome()

# give a variable to the link
# here movies after the 16.12.2023 to 29.02.2023 (today)
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2023-12-16,&primary_language=en"
driver.get(url)

In [9]:
def click_show_more():
    # IMDB uses a Dynaic Show More button at the bottom of page for loading more data this button needs to be
    # clicked repeatedly to load more html data.

    try:
        time.sleep(2)
        show_more_button = driver.find_element(By.XPATH,
                                               "/html/body/div[2]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button")
        driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)

        # adding a delay for the screen to load
        time.sleep(2)

        show_more_button.click()
        return True
    except Exception as e:
        print(f"Error clicking 'show more': {e}")
        return False


num_clicks = 35

# Click the "show more" button multiple times
for _ in range(num_clicks):
    if not click_show_more():
        break

Error clicking 'show more': Message: element click intercepted: Element is not clickable at point (483, 10398)
  (Session info: chrome=122.0.6261.94)
Stacktrace:
0   chromedriver                        0x0000000100be053c chromedriver + 3966268
1   chromedriver                        0x0000000100bd8ac8 chromedriver + 3934920
2   chromedriver                        0x000000010085bda0 chromedriver + 277920
3   chromedriver                        0x00000001008a3e68 chromedriver + 573032
4   chromedriver                        0x00000001008a23ec chromedriver + 566252
5   chromedriver                        0x00000001008a056c chromedriver + 558444
6   chromedriver                        0x000000010089fa9c chromedriver + 555676
7   chromedriver                        0x0000000100894e84 chromedriver + 511620
8   chromedriver                        0x00000001008948a8 chromedriver + 510120
9   chromedriver                        0x00000001008d6bf0 chromedriver + 781296
10  chromedriver          

In [10]:
# the function to extract rating and votes using regular expressions 
# it handles cases where vote counts contain 'K' or 'M' 
# (for thousands and millions), 
# it and returns the extracted values

def get_ratings_votes(exp):
    # Compile a regular expression pattern to match rating and votes
    pattern = re.compile(r'(\d+\.\d+?)\s*\(([^)]+)\)')
    
    # search for the pattern in the input string
    # exp' is a string containing rating and votes in a combined expression
    match = pattern.search(exp)
    
    # define multipliers for handling 'K' and 'M' in vote counts
    multipliers = {"K": 1000, "M": 1000000}
    
    if match:
        # if a match is found:
        if match.group(2)[-1] in multipliers:
            # if the last character of the second group is a multiplier (K or M):
            numeric_part, multiplier = match.group(2)[:-1], match.group(2)[-1] 
            # extract numeric part and multiplier from the second group
            # convert numeric part to float and multiply it with corresponding multiplier
            votes = float(numeric_part) * multipliers[multiplier]
        else:
            # if there's no multiplier, simply convert the second group to float
            votes = float(match.group(2))
        
        # return extracted rating (converted to float) and votes
        return float(match.group(1)), votes
    
    # if no match is found, return "NA" for both rating and votes
    return "NA", "NA"

In [11]:
# loading the HTML data from the current page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# finding all the movie data elements in the HTML
# classes are always updating that is why if code
# does not work, there is a possible 
# ussie in the class of an element
movie_data = soup.findAll('div', attrs={'class': "ipc-metadata-list-summary-item__c"})

# extracting titles of the movies
titles = soup.findAll('h3', attrs={'class': "ipc-title__text"})
titles = [".".join(x.text.split(".")[1:]) for x in titles][:-1]

# initializing empty lists to store movie information
ratings = []
votes = []
metascores = []
years = []
durations = [] 
rated = []
links = []

# class name for extracting ratings
rating_cls = "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"

# iterating through each movie data element
for id, x in enumerate(movie_data):
    # extracting rating information
    rating = x.find('span', attrs={'class': rating_cls}).text if x.find('span', attrs={'class': rating_cls}) else "NA"
    rating, vote = get_ratings_votes(rating)  # Extracting votes from the rating
    ratings.append(rating)  # Storing extracted rating
    votes.append(vote)  # Storing extracted vote count
    
    # extracting metascore if available
    metascore = x.find('span', attrs={"class": "sc-b0901df4-0 bcQdDJ metacritic-score-box"}).text if x.find('span', attrs={"class": "sc-b0901df4-0 bcQdDJ metacritic-score-box"}) else 'NA'
    metascores.append(metascore)  # storing extracted metascore
    
    # extracting metadata (year, duration, and rating)
    metadata = x.find('div', attrs={"class": "sc-be6f1408-7 iUtHEN dli-title-metadata"}) if x.find('div', attrs={"class": "sc-be6f1408-7 iUtHEN dli-title-metadata"}) else 'NA'
    
    # checking if metadata is a string
    if isinstance(metadata, str):
        years.append("NA")
        durations.append("NA")
        rated.append("NA")
    else:
        metadata = [info.text for info in metadata]  # extracting metadata text
        years.append(metadata[0])  # Storing year
        
        # extracting duration if available, otherwise storing "NA"
        try:
            durations.append(metadata[1])
        except:
            durations.append("NA")

        # extracting rating if available, otherwise storing "NA"
        try:
            if metadata[2] == "Not Rated":
                metadata[2] = "NA"
            rated.append(metadata[2])
        except:
            rated.append("NA")
    
    # extracting movie link if available, otherwise storing "NA"
    # this is a crucial part for getting the another part of the data
    # we have to gather all movies links in order to open them and extract data
    link = x.find('a', attrs={"class": "ipc-lockup-overlay ipc-focusable"}).get('href') if x.find('span', attrs={"class": "sc-b0901df4-0 bcQdDJ metacritic-score-box"}) else 'NA'
    links.append(link)  # storing extracted link


In [12]:
#getting the casts, genres and director_list from the other pages
casts = []
genres = []
director_list = []

# open a new weindow
newdriver = webdriver.Chrome()

#loop through each link to gather additional information
for link in links:
    #check if link is available
    if link != 'NA':
        #navigate to the IMDb page for the movie
        newdriver.get("https://www.imdb.com" + link)
        soup = BeautifulSoup(newdriver.page_source, 'html.parser')
        
        #extract genre information
        genre = soup.find('div', attrs={'class':"ipc-chip-list__scroller"})
        # genre = [x.text for x in genre]
        try:
            genres.append(", ".join([x.text for x in genre]))
        except:
            genres.append("NA")
            
        #extract cast information
        cast = soup.findAll('div', attrs={'class': 'sc-bfec09a1-5 hNfYaW' })  
        try:
            casts.append(", ".join([x for x in [x.find('a', attrs={'class': 'sc-bfec09a1-1 gCQkeh'}).text for x in cast][:4]]))
        except:
            casts.append("NA")

        #extract director information
        director = soup.find('div', attrs={'class': 'ipc-metadata-list-item__content-container'}) 
        try:
            director_list.append(director.text) 
        except:
            director_list.append("NA")

    else:
        genres.append("NA")
        casts.append("NA")
        director_list.append("NA")


In [13]:
# create a DataFrame using the extracted movie information
movies_data = pd.DataFrame({
    "Movie Name": titles,        # Column for movie titles
    "Rating": ratings,           # Column for movie ratings
    "Votes": votes,              # Column for number of votes
    "Meta Score": metascores,    # Column for metascore
    "Genre": genres,             # Column for movie genres
    "PG Rating": rated,          # Column for movie ratings (PG)
    "Year": years,               # Column for release year
    "Duration": durations,       # Column for movie duration
    "Cast": casts,               # Column for movie cast
    "Director": director_list   # Column for movie directors
})

# save the DataFrame to a CSV file named "webscrapping_imdb_movie_data_2024.csv"
movies_data.to_csv("New_data.csv", index=True)
